**Get the environment properties**
---

In [43]:
print("\t---Get distro and release information---")
!lsb_release -a 
print("\n\t---Get cpu properties---")
!lscpu | grep -E '^Model name|^Thread|^Core|^Socket|^CPU\('
print("\n\t---Get available memory---")
!free -m -h 
print("\n\t---Get file system attributes---")
!df -h

	---Get distro and release information---
No LSB modules are available.
Distributor ID:	Ubuntu
Description:	Ubuntu 18.04.6 LTS
Release:	18.04
Codename:	bionic

	---Get cpu properties---
CPU(s):              2
Thread(s) per core:  2
Core(s) per socket:  1
Socket(s):           1
Model name:          Intel(R) Xeon(R) CPU @ 2.20GHz

	---Get available memory---
              total        used        free      shared  buff/cache   available
Mem:            12G        1.3G        8.9G        1.3M        2.5G         11G
Swap:            0B          0B          0B

	---Get file system attributes---
Filesystem      Size  Used Avail Use% Mounted on
overlay         108G   25G   84G  23% /
tmpfs            64M     0   64M   0% /dev
shm             5.8G     0  5.8G   0% /dev/shm
/dev/root       2.0G  1.1G  910M  54% /sbin/docker-init
tmpfs           6.4G   72K  6.4G   1% /var/colab
/dev/sda1        41G   27G   14G  67% /etc/hosts
tmpfs           6.4G     0  6.4G   0% /proc/acpi
tmpfs           6.4G

**Requirements installation**
---

In [44]:
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q https://archive.apache.org/dist/spark/spark-3.2.2/spark-3.2.2-bin-hadoop3.2.tgz
!tar xf spark-3.2.2-bin-hadoop3.2.tgz
!pip -qq install findspark
print("Java version: ")
!java -version
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.2.2-bin-hadoop3.2"
!pip -qq install pyspark
!pip -qq install geopandas attrs shapely rtree apache-sedona[spark]


Java version: 
openjdk version "11.0.16" 2022-07-19
OpenJDK Runtime Environment (build 11.0.16+8-post-Ubuntu-0ubuntu118.04)
OpenJDK 64-Bit Server VM (build 11.0.16+8-post-Ubuntu-0ubuntu118.04, mixed mode, sharing)


**Get data from drive**
---

In [45]:
#gets data from csv files in google and puts them in a data folder
!gdown --fuzzy 'https://drive.google.com/file/d/1XqsRpUJJ_Q6wQ-tr4d5QKwGqmP5LLXFh/view?usp=sharing' -O /content/
!unzip -o data.zip 

Downloading...
From: https://drive.google.com/uc?id=1XqsRpUJJ_Q6wQ-tr4d5QKwGqmP5LLXFh
To: /content/data.zip
100% 3.20M/3.20M [00:00<00:00, 118MB/s]
Archive:  data.zip
  inflating: data/CottonYield_1980-2020.csv  
  inflating: data/SurfaceWaterQuality.csv  
  inflating: data/toxicityData/chihuahua_data.csv  
  inflating: data/toxicityData/chihuahua_site.csv  
  inflating: data/toxicityData/coahuila_data.csv  
  inflating: data/toxicityData/coahuila_site.csv  
  inflating: data/toxicityData/durango_data.csv  
  inflating: data/toxicityData/durango_site.csv  
  inflating: data/toxicityData/nuevoleon_data.csv  
  inflating: data/toxicityData/nuevoleon_sites.csv  
  inflating: data/toxicityData/simbology.csv  
  inflating: data/toxicityData/sinaloa_data.csv  
  inflating: data/toxicityData/sinaloa_site.csv  
  inflating: data/toxicityData/sonora_data.csv  
  inflating: data/toxicityData/sonora_site.csv  
  inflating: data/toxicityData/tamaulipas_data.csv  
  inflating: data/toxicityData/tam

**Setup Apache Spark and Sedona**
---

In [46]:
import findspark
findspark.init()
import shapely
import pandas as pd 
import geopandas as gpd
from pyspark.sql import SparkSession
from pyspark import SparkContext
from pyspark import SparkConf
from sedona.register import SedonaRegistrator
from sedona.utils import SedonaKryoRegistrator, KryoSerializer
spark = SparkSession. \
builder. \
appName('GeoBigData'). \
config("spark.serializer", KryoSerializer.getName). \
config("spark.executor.memory", "5g"). \
config("spark.driver.memory", "10g"). \
config('spark.driver.maxResultSize', '5g'). \
config("spark.kryo.registrator", SedonaKryoRegistrator.getName). \
config('spark.jars.packages',
           'org.apache.sedona:sedona-python-adapter-3.0_2.12:1.2.0-incubating,'
           'org.datasyslab:geotools-wrapper:1.1.0-25.2'). \
getOrCreate()
SedonaRegistrator.registerAll(spark)

True

**Load DB**
---

Load the csv files into databases with spark for later use

In [ ]:
DB_SWQ = spark.read.option("header",True).csv(f"/content/data/SurfaceWaterQuality.csv")
DB_SWQ_Q = DB_SWQ.select('CLAVE','SITIO','ESTADO', 'TOX_D_48_UT', 'CALIDAD_TOX_D_48')
DB_cotton = spark.read.option("header",True).csv(f"/content/data/CottonYield_1980-2020.csv")
DB_Cotton_Q = DB_cotton.select('Ano','Estado','Ciclo', 'Modalidad', 'Producccion','Superficie')
DB_ToxChi = spark.read.option("header",True).csv(f"/content/data/toxicityData/chihuahua_data.csv")
DB_ToxChi_Q = DB_ToxChi.select()
DB_ToxCoa = spark.read.option("header",True).csv(f"/content/data/toxicityData/coahuila_data.csv")
DB_ToxDur = spark.read.option("header",True).csv(f"/content/data/toxicityData/durango_data.csv")
DB_ToxNL = spark.read.option("header",True).csv(f"/content/data/toxicityData/nuevoleon_data.csv")
DB_ToxSin = spark.read.option("header",True).csv(f"/content/data/toxicityData/sinaloa_data.csv")
DB_ToxSon = spark.read.option("header",True).csv(f"/content/data/toxicityData/sonora_data.csv")
DB_ToxTam = spark.read.option("header",True).csv(f"/content/data/toxicityData/tamaulipas_data.csv")
DB_ToxTam.show()

+--------------+--------------------+--------------------+----------------------+---------+-------+-------+----+----+----+--------+--------+--------+------+----+-------+-------+-------+-------+-------+-----+-----+-----+-----+-----+------+-----------+---------------+---------------+-----------------+-----------------+----------------+-----------------+-----------------+----------------+-----------+-----------+----------+-----+-------+--------+---------+-------------+------+-----+------------+----------------+----------------+----------------+--------+------------+------------+------------+-------------+-------------+-------------+----+-------+--------+-----------+--------+-----------+--------+-----------+--------------+-------------------+-----------+--------------+-----------------+-------------------+------------+--------------+------+-------------+---------+------+------+-------+--------+-----------+----------------+-------+-----------+-----------+-----------------+------------+-----

# Whatevs